## Script to train a neural network

This script run the training of a neural networks.
For long training, it is advised to used this script as a python script, not the notebook

In [ ]:
import numpy as np
import os
from os.path import join,isdir
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from neuralsw.model.modeltools import mymodel,loadmymodel
from keras import regularizers
from keras.optimizers import SGD
import glob
import warnings
import matplotlib.pyplot as plt

In [ ]:
# Initialize parameters
PLOT = True
SAVE = True #to save plots
#rootdir
rootdir = os.path.realpath(\
    os.path.join(os.getcwd(),'../..'))

#directory to store the data
datadir = os.path.realpath(os.path.join(rootdir,'data'))

#training directory
#traindir = 'train_uparam_nonoise_std'
#traindir = 'train_uparam_noise01_std'
traindir = 'train_vparam_nonoise_std'
#traindir = 'train_vparam_noise01_std'

#First guess net
#fgnet = None
fgnet_id = None

#neural net identifier (to personalize the output name)
nnid = '0'

keywords = traindir.split('_')

#param
param = keywords[1]

#files containing inputs/outputs
Xfile = join(datadir,traindir,'data_X.npy')
yfile = join(datadir,traindir,'data_y.npy')

#input name
if fgnet_id is not None:
    fgnet = '_'.join(['nn'+fgnet_id,param,keywords[2],keywords[3]])

    #Raise warnings if there are some issues with the selected names
    if not os.path.isdir(join(datadir,fgnet)):
        warnings.warn(join(datadir,fgnet) +' does not exist')
    print('first guess neural net:',fgnet)

    if fgnet_id == nnid:
        warnings.warn ('first guess and output have the same id')

#output name
netname = '_'.join(['nn'+nnid,param,keywords[2],keywords[3]])
print('output name of the neural net:',netname)


In [ ]:
## Load the data
X = np.load(Xfile)
y = np.load(yfile)
nt,ny,nx,npar = X.shape



In [ ]:
## Define the nn model
if fgnet_id is not None:
    pklfile=glob.glob(join(datadir,fgnet,'*.pkl'))[0]
    nn = loadmymodel(pklfile)
else:
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
              padding='same',
              #kernel_regularizer=regularizers.l1(0.001),
              input_shape=(ny, nx, npar)))
    #model.add(Dropout(0.2))
    model.add(Conv2D(1, (1, 1), activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')

    #normalization
    moy = np.zeros(npar)
    et = np.zeros(npar)
    for j in range(npar):
            moy[j] = np.mean(X[:,:,:,j].ravel())
            et[j] = np.std(X[:,:,:,j].ravel())
          
    moy_y = np.mean(y.ravel())
    et_y = np.std(y.ravel())
        
    nn = mymodel(model,moyX=moy,etX=et,moyY=moy_y,etY=et_y)




In [ ]:
## Training
nn.fit(X, y, epochs=200, batch_size=1,validation_split=0.1)

In [ ]:
## Save the neural net
if not isdir(join(datadir, netname)):
    os.mkdir(join(datadir, netname))
nn.save(join(datadir,netname,'model_'+param+'.pkl'))


In [ ]:
## Plots and save
if PLOT:
    #plot history
    plt.semilogy(nn._history['loss'], color='gray',label='train')
    plt.semilogy(nn._history['val_loss'], color='black',label='test')
    plt.legend()
    if SAVE:
        plt.savefig(join(datadir, netname, 'history.png'))
    plt.show()
    

In [ ]:
## a small scatter plot to check
mini = min(y.ravel())
maxi = max(y.ravel())

y_predict = nn.predict(X)
plt.plot(y.ravel(),y_predict.ravel(),'.k')
plt.plot([mini,maxi],[mini,maxi],'r-')
plt.xlabel(param + ' true')
plt.ylabel(param + ' nn')
plt.show()